In [3]:
import d2lzh as d2l
from mxnet import autograd,gluon,init,nd
from mxnet.gluon import loss as gloss,nn
def drop_out(X,drop_prob):
    assert 0<=drop_prob<=1
    keep_prob=1-drop_prob
    if keep_prob==0:
        return X.zeros_like()
    mask=nd.random.uniform(0,1,X.shape)<keep_prob
    return mask*X/keep_prob

In [27]:
X=nd.arange(16).reshape(2,8)
Y=drop_out(X,1)
Y


[[0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]]
<NDArray 2x8 @cpu(0)>

In [28]:
Y.mean()


[0.]
<NDArray 1 @cpu(0)>

In [30]:
num_inputs,num_outputs,num_hiddens1,num_hiddens2=784,10,256,256
W1=nd.random.normal(scale=0.01,shape=(num_inputs,num_hiddens1))
b1=nd.zeros(num_hiddens1)
W2=nd.random.normal(scale=0.01,shape=(num_hiddens1,num_hiddens2))
b2=nd.zeros(num_hiddens2)
W3=nd.random.normal(scale=0.01,shape=(num_hiddens2,num_outputs))
b3=nd.zeros(num_outputs)
params=[W1,b1,W2,b2,W3,b3]
for param in params:
    param.attach_grad()

In [39]:
drop_prob1,drop_prob2=0.5,0.2
def net(X):
    X=X.reshape((-1,num_inputs))
    H1=(nd.dot(X,W1)+b1).relu()
    if autograd.is_training():
        H1=drop_out(H1,drop_prob1)
    H2=(nd.dot(H1,W2)+b2).relu()
    if autograd.is_training():
        H2=drop_out(H2,drop_prob2)
    return nd.dot(H2,W3)+b3

In [40]:
num_epochs,lr,batch_size=5,0.5,256
loss=gloss.SoftmaxCrossEntropyLoss()
train_iter,test_iter=d2l.load_data_fashion_mnist(batch_size)
d2l.train_ch3(net,train_iter,test_iter,loss,num_epochs,batch_size,params,lr)

epoch 1, loss 0.3685, train acc 0.863, test acc 0.875
epoch 2, loss 0.3609, train acc 0.867, test acc 0.877
epoch 3, loss 0.3543, train acc 0.870, test acc 0.883
epoch 4, loss 0.3467, train acc 0.872, test acc 0.880
epoch 5, loss 0.3421, train acc 0.874, test acc 0.879


In [35]:
net=nn.Sequential()
net.add(nn.Dense(256,activation="relu"),nn.Dropout(drop_prob1),
       nn.Dense(256,activation="relu"),nn.Dropout(drop_prob2),
       nn.Dense(10))
net.initialize(init.Normal(sigma=0.01))

In [36]:
trainer=gluon.Trainer(net.collect_params(),'sgd',{'learning_rate':lr})
d2l.train_ch3(net,train_iter,test_iter,loss,num_epochs,batch_size,None,None,trainer)

epoch 1, loss 1.1766, train acc 0.556, test acc 0.742
epoch 2, loss 0.5807, train acc 0.784, test acc 0.839
epoch 3, loss 0.4892, train acc 0.821, test acc 0.851
epoch 4, loss 0.4440, train acc 0.837, test acc 0.860
epoch 5, loss 0.4211, train acc 0.846, test acc 0.861


In [46]:
net=nn.Sequential()
net.add(nn.Dense(10))
net.initialize(init.Normal(sigma=0.01))
wd=0.00001
trainer=gluon.Trainer(net.collect_params(),'sgd',{'learning_rate':lr,'wd':wd})
d2l.train_ch3(net,train_iter,test_iter,loss,num_epochs,batch_size,None,None,trainer)

epoch 1, loss 1.6856, train acc 0.714, test acc 0.806
epoch 2, loss 1.0182, train acc 0.783, test acc 0.807
epoch 3, loss 0.9471, train acc 0.796, test acc 0.846
epoch 4, loss 0.8623, train acc 0.805, test acc 0.823
epoch 5, loss 0.8376, train acc 0.808, test acc 0.847
